In [1]:
%load_ext autoreload
%autoreload 2
# %cd /p/mmcardiac/Jerry/code/CINE-DENSE-Translation
%cd /home/jx8fh/Projects/LaMoD
import numpy as np
import torch
import matplotlib.pyplot as plt
from pathlib import Path
from modules.data import check_dict

/sfs/gpfs/tardis/home/jx8fh/Projects/LaMoD


In [2]:
# 1. cine-DENSE paired data
cine_DENSE_fname = '/scratch/jx8fh/2024-04-21-matched-cine-DENSE-from-Yu-all-with-resampled.npy'
cine_DENSE_data = np.load(cine_DENSE_fname, allow_pickle=True).tolist()

In [3]:
print(f"length of data: {len(cine_DENSE_data)}")
check_dict(cine_DENSE_data[0])

length of data: 105
AnalysisInfo                                                 dict_keys(['ResampleMethod', 'ResampleDistance', 'SpatialSmoothing', 'TemporalOrder', 'Xseed', 'Yseed', 'Zseed', 'FramesForAnalysis', 'Nmodel', 'PositionA', 'PositionB', 'Clockwise'])
AnalysisInstanceUID                                          1.3.6.1.4.1.9590.100.1.2.64946301013757307506911904523491151334
DENSEInfo                                                    dict_keys(['Name', 'UID', 'Type', 'MagIndex', 'PhaIndex', 'Number', 'PixelSpacing', 'Scale', 'EncFreq', 'SwapFlag', 'NegFlag'])
DENSE_Eul_displacement_field_Multipliers                     (3,)                
DENSE_Eul_displacement_field_X                               (48, 48, 51)        
DENSE_Eul_displacement_field_Y                               (48, 48, 51)        
DENSE_Lag_displacement_field_X                               (48, 48, 51)        
DENSE_Lag_displacement_field_Y                               (48, 48, 51)        
DENSE_frame

In [4]:
# 2. cine prediction (disp and strain)
# load previous experimnents results
data_recompute_strain_fname_prev = '/scratch/jx8fh/2024-06-22-MICCAI-ShapeMI-Workshop-data/2024-05-04-matched-cine-DENSE-from-Yu-all-recompute-strain-DIL2.npy'
# data_recompute_strain_fname_prev = '/p/miauva/data/Jerry/medical-images/Cardiac-FromKen/Yu-cine-DENSE-data/2024-05-04-matched-cine-DENSE-from-Yu-all-recompute-strain-DIL2.npy'
data_recompute_strain_prev = np.load(data_recompute_strain_fname_prev, allow_pickle=True).tolist()
check_dict(data_recompute_strain_prev[0])

CCImg                                                        (48, 48, 51)        
CCImg_GT                                                     (128, 128, 51)      
DENSE_strain                                                 dict_keys(['origin', 'insertion', 'CCs4', 'CCs6', 'CCs18', 'Seg4', 'Seg6', 'Seg18', 'layerid_map', 'CCs4_endo', 'CCs6_endo', 'CCs18_endo', 'CCs4_mid', 'CCs6_mid', 'CCs18_mid', 'CCs4_epi', 'CCs6_epi', 'CCs18_epi', 'CCs4Resampled', 'CCs4_endoResampled', 'CCs4_epiResampled', 'CCs4_midResampled', 'CCs6Resampled', 'CCs6_endoResampled', 'CCs6_epiResampled', 'CCs6_midResampled', 'CCs18Resampled', 'CCs18_endoResampled', 'CCs18_epiResampled', 'CCs18_midResampled'])
Diffusion_cine_strain                                        dict_keys(['origin', 'insertion', 'CCs4', 'CCs6', 'CCs18', 'Seg4', 'Seg6', 'Seg18', 'CCs4Resampled', 'CCs6Resampled', 'CCs18Resampled'])
UNetR_cine_strain                                            dict_keys(['origin', 'insertion', 'CCs4', 'CCs6', 'CCs1

In [5]:
len(data_recompute_strain_prev)

105

In [6]:
# 3. which cases to check?
# debug/notebooks/2025-01-06-plot-all-cine-strain-curves.ipynb
# 45  84  43  51  81  82
# 7   37  92  102 82  45

In [7]:
from IPython.display import HTML
from matplotlib import animation
from matplotlib import rc
from IPython.display import HTML
rc('animation', html='jshtml')

In [8]:
import copy
def get_disp_mask(disp_X, disp_Y, thres=5e-1):
    norm = np.sqrt(disp_X**2 + disp_Y**2)
    mask = norm > thres
    return mask

def apply_mask_to_displacement(displacement, mask_thres=5e-1):
    """
    Deep copy the input displacement data and make out-of-mask regions NaN.
    
    :param displacement: numpy array of shape (2, T, H, W)
    :return: masked displacement array of the same shape
    """
    masked_displacement = copy.deepcopy(displacement)
    
    # Compute mask for a middle time frame
    middle_frame = displacement.shape[1] // 2
    mask = get_disp_mask(displacement[0, middle_frame], displacement[1, middle_frame], thres=mask_thres)
    
    # Apply mask to all time frames
    mask_4d = np.tile(mask[None, None, :, :], (2, displacement.shape[1], 1, 1))
    masked_displacement[~mask_4d] = np.nan
    
    return masked_displacement

def get_visualization_window(displacement, margin=10):
    """
    Choose the proper visualization window based on the non-NaN region.
    
    :param displacement: numpy array of shape (2, T, H, W)
    :param margin: number of pixels to add as margin around the non-NaN region
    :return: tuple of (x_min, x_max, y_min, y_max) for the visualization window
    """
    # Use a middle time frame to determine the non-NaN region
    middle_frame = displacement.shape[1] // 2
    mask = ~np.isnan(displacement[0, middle_frame])
    
    non_nan_y, non_nan_x = np.where(mask)
    
    x_min, x_max = non_nan_x.min(), non_nan_x.max()
    y_min, y_max = non_nan_y.min(), non_nan_y.max()
    
    # Add margin
    x_min = max(0, x_min - margin)
    x_max = min(displacement.shape[3], x_max + margin)
    y_min = max(0, y_min - margin)
    y_max = min(displacement.shape[2], y_max + margin)
    
    return x_min, x_max, y_min, y_max

In [9]:
def create_image_gif(image_video, output_path, fps=10, subsample_rate=1, x_min=None, x_max=None, y_min=None, y_max=None):
    """
    Create a GIF from an image video.
    
    :param image_video: numpy array of shape (H, W, T)
    :param output_path: path to save the output GIF
    :param fps: frames per second for the output GIF
    :param subsample_rate: rate at which to subsample the images
    :param x_min, x_max, y_min, y_max: visualization window parameters
    """
    fig, ax = plt.subplots(figsize=(10, 10), constrained_layout=True)
    im = ax.imshow(image_video[::subsample_rate, ::subsample_rate, 0], cmap='gray')
    ax.axis('off')
    ax.invert_yaxis()
    
    if x_min is not None and x_max is not None and y_min is not None and y_max is not None:
        ax.set_xlim(x_min // subsample_rate, x_max // subsample_rate)
        ax.set_ylim(y_max // subsample_rate, y_min // subsample_rate)  # Reversed for imshow
    
    def animate(i):
        im.set_data(image_video[::subsample_rate, ::subsample_rate, i])
        return (im,)

    anim = animation.FuncAnimation(fig, animate, frames=image_video.shape[2], interval=100, blit=True)
    anim.save(output_path, writer='ffmpeg', fps=fps)
    plt.close(fig)

def create_displacement_gif(displacement_video, output_path, fps=10, subsample_rate=1, quiver_scale=1, quiver_width=None, x_min=None, x_max=None, y_min=None, y_max=None):
    """
    Create a GIF from a displacement video.
    
    :param displacement_video: numpy array of shape (2, T, H, W)
    :param output_path: path to save the output GIF
    :param fps: frames per second for the output GIF
    :param subsample_rate: rate at which to subsample the displacement field
    :param quiver_scale: scale for the quiver plot
    :param x_min, x_max, y_min, y_max: visualization window parameters
    """
    fig, ax = plt.subplots(figsize=(10, 10), constrained_layout=True)
    ax.set_facecolor('black')
    quiver = ax.quiver(displacement_video[0, 0, ::subsample_rate, ::subsample_rate],
                       displacement_video[1, 0, ::subsample_rate, ::subsample_rate],
                       scale=quiver_scale, units='xy', color='y', width=quiver_width)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal')
    
    if x_min is not None and x_max is not None and y_min is not None and y_max is not None:
        ax.set_xlim(x_min // subsample_rate, x_max // subsample_rate)
        ax.set_ylim(y_min // subsample_rate, y_max // subsample_rate)

    def animate(i):
        quiver.set_UVC(displacement_video[0, i, ::subsample_rate, ::subsample_rate],
                       displacement_video[1, i, ::subsample_rate, ::subsample_rate])
        return (quiver,)

    anim = animation.FuncAnimation(fig, animate, frames=displacement_video.shape[1], interval=100, blit=True)
    anim.save(output_path, writer='ffmpeg', fps=fps)
    plt.close(fig)

def create_combined_gif(image_video, displacement_video, output_path, fps=10, subsample_rate=1, quiver_scale=1, quiver_width=None, x_min=None, x_max=None, y_min=None, y_max=None):
    fig, ax = plt.subplots(figsize=(10, 10), constrained_layout=True)
    im = ax.imshow(image_video[::subsample_rate, ::subsample_rate, 0], cmap='gray')
    ax.invert_yaxis()
    quiver = ax.quiver(displacement_video[0, 0, ::subsample_rate, ::subsample_rate],
                       displacement_video[1, 0, ::subsample_rate, ::subsample_rate],
                       scale=quiver_scale, units='xy', color='y', width=quiver_width)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal')
    
    if x_min is not None and x_max is not None and y_min is not None and y_max is not None:
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)

    def animate(i):
        im.set_data(image_video[::subsample_rate, ::subsample_rate, i])
        quiver.set_UVC(displacement_video[0, i, ::subsample_rate, ::subsample_rate],
                       displacement_video[1, i, ::subsample_rate, ::subsample_rate])
        return (im, quiver)

    anim = animation.FuncAnimation(fig, animate, frames=image_video.shape[2], interval=100, blit=True)
    anim.save(output_path, writer='ffmpeg', fps=fps)
    plt.close(fig)

In [10]:
def create_gif(image_video=None, displacement_video=None, output_path='output.gif', fps=10, subsample_rate=1, quiver_scale=1, quiver_width=None, mask_thres=5e-1):
    """
    Create a GIF based on the provided inputs.
    
    :param image_video: numpy array of shape (H, W, T) or None
    :param displacement_video: numpy array of shape (2, T, H, W) or None
    :param output_path: path to save the output GIF
    :param fps: frames per second for the output GIF
    :param subsample_rate: rate at which to subsample the images and displacement field
    :param quiver_scale: scale for the quiver plot
    """
    if displacement_video is not None:
        displacement_video = apply_mask_to_displacement(displacement_video, mask_thres=mask_thres)
        x_min, x_max, y_min, y_max = get_visualization_window(displacement_video)
    else:
        x_min, x_max, y_min, y_max = None, None, None, None
    
    if image_video is not None and displacement_video is not None:
        create_combined_gif(image_video, displacement_video, output_path, fps, subsample_rate, quiver_scale, quiver_width, x_min, x_max, y_min, y_max)
    elif image_video is not None:
        create_image_gif(image_video, output_path, fps, subsample_rate, x_min, x_max, y_min, y_max)
    elif displacement_video is not None:
        create_displacement_gif(displacement_video, output_path, fps, subsample_rate, quiver_scale, quiver_width, x_min, x_max, y_min, y_max)
    else:
        raise ValueError("At least one of image_video or displacement_video must be provided.")

In [14]:
# make the plots
import copy
disp_GT_scale = 1
disp_pred_scale = 1
save_dir = Path('debug/notebooks/2025-01-24-make-paired-cine-DENSE-video-for-SCMR-LaMoD-presentation')
for datum_idx in [45]:
    slice_cine_mag = cine_DENSE_data[datum_idx]['cine_cropped_images_rescaled_DENSE48']
    slice_cine_disp_pred = copy.deepcopy(data_recompute_strain_prev[datum_idx]['MotionDecoder1x1_cine_disp'])
    slice_DENSE_disp_GT = copy.deepcopy(np.moveaxis(np.concatenate(
        [
            cine_DENSE_data[datum_idx]['DENSE_Lag_displacement_field_X'][None],
            cine_DENSE_data[datum_idx]['DENSE_Lag_displacement_field_Y'][None],
        ],
        axis=0
    ), -1, 1))
    slice_DENSE_disp_GT_subsampled = slice_DENSE_disp_GT[:,::3]

    slice_cine_disp_pred[:, :, ::2,::2] = np.nan
    slice_DENSE_disp_GT[:, :, ::2,::2] = np.nan

    # cine magnituide images
    # create_gif(
    #     image_video=slice_cine_mag,
    #     displacement_video=None,
    #     output_path=save_dir / f'case{datum_idx:03d}-cine-mag.gif',
    #     mask_thres=1e-3,
    #     quiver_scale=disp_pred_scale
    # )

    # cine displacement field prediction
    create_gif(
            image_video=None,
            displacement_video=slice_cine_disp_pred,
            output_path=save_dir / f'case{datum_idx:03d}-cine-disp-subsampled.gif',
            mask_thres=1e-3,
            quiver_scale=disp_pred_scale,
            quiver_width=0.2
        )
    
    # cine magnitude images + displacement field
    create_gif(
            image_video=slice_cine_mag,
            displacement_video=slice_cine_disp_pred,
            output_path=save_dir / f'case{datum_idx:03d}-cine-mag-disp-subsampled.gif',
            mask_thres=1e-3,
            quiver_scale=disp_pred_scale,
            quiver_width=0.2
        )

    # DENSE displacement field
    # create_gif(
    #     image_video=None,
    #     displacement_video=slice_DENSE_disp_GT_subsampled,
    #     output_path=save_dir / f'case{datum_idx:03d}-DENSE-disp-subsampled.gif',
    #     mask_thres=1e-3,
    #     quiver_scale=disp_GT_scale,
    #     quiver_width=0.1
    # )




In [12]:
slice_cine_disp_pred.shape

(2, 25, 48, 48)

In [13]:
slice_DENSE_disp_GT.shape

(2, 57, 48, 48)